In [ ]:
# 국룰 세팅1
!pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 37.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=6d7a7c93606fe4b36ec02f726898a7375d910bc8ad2dfea2d88a9b3b27b83651
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
# 국룰 세팅2
import pandas as pd
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()

In [ ]:
# 맥은 가끔 스파크세션이 제대로 생성되지 않아 확인을 한번 해줬다.
spark

In [ ]:
# 국룰 세팅3
import pyspark.sql.types as T
import pyspark.sql.functions as F

<h1>위치데이터 준비

In [ ]:
station_locate = spark.read.format("csv").option("header", True).load("/content/drive/MyDrive/data/station_coordinate.csv", encoding='utf-8')
station_locate = station_locate.withColumnRenamed("name", "역명")
station_locate = station_locate.withColumnRenamed("line", "노선명")
station_locate = station_locate.select('노선명', '역명', 'lat', 'lng').distinct()

병기역명 처리

In [ ]:
from itertools import chain

역명이 다른 case처리

In [ ]:
# 해당하는 역들로 dictionary를 만들고 그 key값들을 list에 저장
change_name = {'서울': '서울역', '419민주묘지': '4.19민주묘지', '지제': '평택지제'}
change_name_list = []
for key in change_name:
    change_name_list.append(key)

In [ ]:
# 지난번에 설명을 덜한부분 보충 설명(A to Z)
# 해당 작업(col 맵핑?)은 pandas로 하는것을 추천. spark 3.0이상을 사용하면 이렇게 해야함.
# dictionary에 items()를 사용하면 (key, value)로 이루어진 list를 반환
# list앞에 *를 붙이면 list의 원소들을 unpacking해준다. 이 경우에는 (key, value)로 이루어진 tuple들이 반환된다.
# chain() : 해당 iterable들 내의 모든 iterable이 소진될 때 까지 순회함. 위의 tuple들을 차례로 순회하며 key와 value를 반환
# lit() : literal한 값들로 이루어진 col을 생성한다.
# creat_map() : key-value 쌍들로 이루어진 col을 생성한다. (key => value, key => value) 이런 느낌
mapping_expr = F.create_map([F.lit(x) for x in chain(*change_name.items())])
station_locate = station_locate.withColumn("역명", F.when(station_locate.역명.isin(change_name_list), mapping_expr[F.col("역명")]).otherwise(station_locate.역명))

In [ ]:
# 참고
print(*change_name.items())

('서울', '서울역') ('419민주묘지', '4.19민주묘지') ('지제', '평택지제')


In [ ]:
# 참고
print([F.lit(x) for x in chain(*change_name.items())])

[Column<'서울'>, Column<'서울역'>, Column<'419민주묘지'>, Column<'4.19민주묘지'>, Column<'지제'>, Column<'평택지제'>]


In [ ]:
# 참고
mapping_expr

Column<'map(서울, 서울역, 419민주묘지, 4.19민주묘지, 지제, 평택지제)'>

잘못된 위치 데이터 갱신

In [ ]:
# 인천공항1터미널
station_locate = station_locate.withColumn("lat", F.when(station_locate.역명=='인천공항1터미널', '37.447492').otherwise(station_locate.lat))
station_locate = station_locate.withColumn("lng", F.when(station_locate.역명=='인천공항1터미널', '126.452555').otherwise(station_locate.lng))
# 인천공항2터미널
station_locate = station_locate.withColumn("lat", F.when(station_locate.역명=='인천공항2터미널', '37.467407').otherwise(station_locate.lat))
station_locate = station_locate.withColumn("lng", F.when(station_locate.역명=='인천공항2터미널', '126.434142').otherwise(station_locate.lng))
# 경기광주역
station_locate = station_locate.withColumn("lat", F.when(station_locate.역명=='경기광주', '37.398775').otherwise(station_locate.lat))
station_locate = station_locate.withColumn("lng", F.when(station_locate.역명=='경기광주', '127.252308').otherwise(station_locate.lng))

# 신촌역과 양평역은 동일한 이름의 역명이 2개 존재해서 위치 데이터가 잘못기록되어 있었다.
# 신촌역(경의중앙)
station_locate = station_locate.withColumn("lat", F.when((station_locate.역명=='신촌') & (station_locate.노선명=='경의선'), '37.559864').otherwise(station_locate.lat))
station_locate = station_locate.withColumn("lng", F.when((station_locate.역명=='신촌') & (station_locate.노선명=='경의선'), '126.942571').otherwise(station_locate.lng))
# 양평역(서울)
station_locate = station_locate.withColumn("lat", F.when((station_locate.역명=='양평') & (station_locate.노선명=='05호선'), '37.525338').otherwise(station_locate.lat))
station_locate = station_locate.withColumn("lng", F.when((station_locate.역명=='양평') & (station_locate.노선명=='05호선'), '126.886194').otherwise(station_locate.lng))

In [ ]:
# 가짜 신촌역 이름 바꿔줌
station_locate = station_locate.withColumn("역명", F.when((station_locate.역명=='신촌') & (station_locate.노선명=='경의선'), '신촌(경의중앙)').otherwise(station_locate.역명))

In [ ]:
# 양평역 이름 바꿔줌
station_locate = station_locate.withColumn("역명", F.when((station_locate.역명=='양평') & (station_locate.노선명=='05호선'), '양평(서울)').otherwise(station_locate.역명))

In [ ]:
station_locate = station_locate.select('역명', 'lat', 'lng').distinct()

In [ ]:
# 양평이랑 신촌역 위치반영 제대로 되었는지 확인
station_locate.where(F.col('역명')=='양평(서울)').show()

+----------+---------+----------+
|      역명|      lat|       lng|
+----------+---------+----------+
|양평(서울)|37.525338|126.886194|
+----------+---------+----------+



In [ ]:
station_locate.where(F.col('역명')=='양평').show()

+----+---------+----------+
|역명|      lat|       lng|
+----+---------+----------+
|양평|37.492832|127.491814|
+----+---------+----------+



<h1>역별 이용자수 차이 데이터 준비

In [ ]:
df = spark.read.format("parquet").option("header", True).load("/content/drive/MyDrive/data/중간저장/preprocessing")
df

DataFrame[사용일자: date, 노선명: string, 역명: string, 승차총승객수: int, 하차총승객수: int, 이용객수: int, 코로나: int]

In [ ]:
# 아까 동일한 역명이 존재함을 확인했음. 여기서도 확인해봄
df.where(F.col('역명')=='양평').show()

+----------+----------+----+------------+------------+--------+------+
|  사용일자|    노선명|역명|승차총승객수|하차총승객수|이용객수|코로나|
+----------+----------+----+------------+------------+--------+------+
|2018-05-10|     5호선|양평|        8009|        8100|   16109|    -1|
|2018-05-10|경의중앙선|양평|        3800|        3856|    7656|    -1|
|2018-05-11|     5호선|양평|        8246|        8137|   16383|    -1|
|2018-05-11|경의중앙선|양평|        4248|        4422|    8670|    -1|
|2018-05-12|경의중앙선|양평|        3395|        3577|    6972|    -1|
|2018-05-12|     5호선|양평|        4264|        4115|    8379|    -1|
|2018-05-13|     5호선|양평|        3195|        3196|    6391|    -1|
|2018-05-13|경의중앙선|양평|        4853|        4803|    9656|    -1|
|2018-05-14|     5호선|양평|        8045|        7917|   15962|    -1|
|2018-05-14|경의중앙선|양평|        3921|        3885|    7806|    -1|
|2018-05-15|     5호선|양평|        8215|        8226|   16441|    -1|
|2018-05-15|경의중앙선|양평|        3913|        3855|    7768|    -1|
|2018-05-16|경의중앙선|양평|        

신촌역이 2개 존재하므로 경의중앙선 신촌역을 신촌(경의중앙)으로 변경해줌<br>
마찬가지로 5호선 양평역을 양평(서울)로 변경해줌<br>
이름을 바꿔주지 않으면 나중에 역명 col을 이용해 join할 때 문제가 생긴다.

In [ ]:
df = df.withColumn("역명", F.when((df.역명=='신촌') & (df.노선명=='경의중앙선'), '신촌(경의중앙)').otherwise(df.역명))
df = df.withColumn("역명", F.when((df.역명=='양평') & (df.노선명=='5호선'), '양평(서울)').otherwise(df.역명))

In [ ]:
# 위치 데이터가 얼추 준비되었으므로 다시 역별 이용자수 데이터로 돌아옴.
# 코로나 이전과 이후의 이용자 차이를 sum함수를 구하기위해 -1과 1로 구분된 '코로나' 칼럼을 이용함
# 역별이용자수 칼럼은 코로나 이전의 이용자수를 음수로, 코로나 이후의 이용자수를 양수로 저장했다.
df2 = df.withColumn('역별이용자수', F.col("이용객수")*F.col("코로나"))

In [ ]:
# sum을 이용해 코로나 이전과 이후의 역별이용자수차이를 구해주어 '이용자수차이' 칼럼에 저장한다.
df2 = df2.groupby('`역명`').agg(F.sum('`역별이용자수`').alias('이용자수차이'))

In [ ]:
# 코로나 전후 620일의 데이터이므로 620으로 나누어 '일간이용자수차이' 칼럼에 저장한다.
df2 = df2.withColumn('일간이용자수차이', (F.col('이용자수차이')/620).cast(T.IntegerType())).orderBy('역명')

<h1>노선별 역 순서 데이터</h1>

In [ ]:
# 우리가 다루고 있는 노선정보들을 확인하여 순서 데이터의 노선명을 수정해줌(물리)
df.select('노선명').distinct().show()

+----------+
|    노선명|
+----------+
|우이신설선|
|     7호선|
|     4호선|
|     1호선|
|     3호선|
|    경강선|
|    경춘선|
|     9호선|
|     6호선|
|경의중앙선|
|     5호선|
|     8호선|
|  공항철도|
|     2호선|
|수인분당선|
+----------+



또한 아래의 사유중, 운행을 시작한지 얼마되지않아 비교에서 제외할 역들을 제거해준다.(물리)<br>

|역명|이유|
|--|--|
|                   강일|    2021년 3월 27일부터 운행하였다.|
|                   검암|실제로 검암역을 지나지 않는 경의중앙선이 집계가 되어있었다.|
|                   계양|실제로 계양역을 지나지 않는 경의중앙선이 집계가 되어있었다.|
|                   고색|2020년 9월 12일부터 운행하였다.|
|                   공덕|경의선과 중앙선을 경의중앙선으로 통합하면서 차이가 발생했다.|
|               김포공항|실제로 김포공항역을 지나지 않는 경의중앙선이 집계가 되어있었다.|
|                 낙성대|역명이 변경되었다.|
|         낙성대(강감찬)|역명이 변경되었다.|
|     동대문역사문화공원|역명이 변경되었다.|
|동대문역사문화공원(DDP)|역명이 변경되었다.|
|               둔촌오륜|2018년 12월 1일부터 운행하였다.|
|       디지털미디어시티|경의선과 중앙선을 경의중앙선으로 통합하면서 차이가 발생했다.|
|               마곡나루|역명이 변경되었다. + 2018년 9월 29일부터 공항철도 노선이 추가로 개통되었다.|
|   마곡나루(서울식물원)|역명이 변경되었다. + 2018년 9월 29일부터 공항철도 노선이 추가로 개통되었다.|
|                   미사|2020년 8월 8일부터 운행하였다.|
|                   복정|수인선과 분당선을 수인분당선으로 통합하면서 차이가 발생하였다.|
|                   부발|2020년 8월 3월에 큰 폭우가 내려 운행이 잠시 중단되었었다.|
|                   사리|2020년 9월 12일부터 운행을 시작하였다.|
|                   삼전|2018년 12월 1일부터 운행을 시작하였다.|
|                 서울역|경의선과 중앙선을 경의중앙선으로 통합하면서 차이가 발생했다.|
|                   석촌|2018년 12월 1일부터 운행을 시작하였다.|
|               석촌고분|2018년 12월 1일부터 운행을 시작하였다.|
|             세종대왕릉|2020년 8월 3월에 큰 폭우가 내려 운행이 잠시 중단되었었다.|
|               송파나루|2018년 12월 1일부터 운행을 시작하였다.|
|                   신내|2019년 12월 21일부터 6호선 노선이 추가로 개통되었다.|
|                   야목|2020년 9월 12일부터 운행을 시작하였다.|
|                   어천|2020년 9월 12일부터 운행을 시작하였다.|
|                 연신내|이 역은 개찰구가 따로 마련되어 있지 않기때문에 모든 승하차인원이 3호선으로 집계된다. 때문에 6호선으로 집계되는 항목을 따로 처리해줘야한다.|
|                 오목천|2020년 9월 12일부터 운행을 시작하였다.|
|   올림픽공원(한국체대)|2018년 12월 1일부터 9호선 노선이 추가로 개통되었다.|
|                 용마산|역명이 변경되었다.|
|   용마산(용마폭포공원)|역명이 변경되었다.|
|                 임진강|2020년 3월 28일부터 운행을 시작하였다.|
|           중앙보훈병원|2018년 12월 1일부터 운행을 시작하였다.|
|                   지제|역명이 변경되었다.|
|                   지축|일산선 노선을 3호선으로 편입하면서 차이가 발생하였다.|
|                   창동|이 역은 개찰구가 따로 마련되어 있지 않기때문에 모든 승하차인원이 4호선으로 집계된다. 때문에 1호선으로 집계되는 항목을 따로 처리해줘야한다.|
|                 충무로|이 역은 개찰구가 따로 마련되어 있지 않기때문에 모든 승하차인원이 4호선으로 집계된다. 때문에 3호선으로 집계되는 항목을 따로 처리해줘야한다.|
|               평택지제|역명이 변경되었다.|
|             하남검단산|2021년 3월 27일부터 운행을 시작하였다.|
|    하남시청(덕풍·신장)|2021년 3월 27일부터 운행을 시작하였다.|
|               하남풍산|2020년 8월 8일부터 운행을 시작하였다.|
|               한성백제|2018년 12월 1일부터 운행을 시작하였다.|
|               홍대입구|경의선과 중앙선을 경의중앙선으로 통합하면서 차이가 발생했다.|

In [ ]:
ordered = spark.read.format("csv").option("header", True).load("/content/drive/MyDrive/data/ordered_line(ver2).csv", encoding='utf-8')

In [ ]:
ordered

DataFrame[노선명: string, 노선명(sub): string, 역명: string, 순번: string]

기본적인 데이터프레임이 모두 준비되었으므로 join을 해보자.

In [ ]:
# 먼저 역별 이용자수차이를 저장한 데이터프레임과 역별 위치를 기록한 데이터프레임을 합침
# 이 두 데이터프레임은 앞서 다른 세션에서 역명을 다 통일했으므로 문제가 없을것으로 기대.
df = df2.join(station_locate, on='역명', how='left')

In [ ]:
# 널값 확인
df.where(F.col('lat').isNull()==True).show()

+----+------------+---+---+
|역명|이용자수차이|lat|lng|
+----+------------+---+---+
+----+------------+---+---+



이제 노선별 순서를 기록한 데이터프레임과 앞에서 합친 데이터 프레임을 합쳐보자<br>
아마 두 데이터프레임의 역명이 달라 널값이 생길것이다.

In [ ]:
ordered_df = ordered.join(df, on="역명", how='left')
ordered_df.select('역명').where(F.col('이용자수차이').isNull()==True).distinct().count()

80

...63개일줄 알았는데 80개가 나왔다.

In [ ]:
# 어떤역들이 있는지 확인.
ordered_df.select('역명').where(F.col('이용자수차이').isNull()==True).distinct().show(80)

+------------------+
|              역명|
+------------------+
|       419민주묘지|
|              강변|
|              강일|
|            경복궁|
|            고려대|
|              고색|
|              공릉|
|            광나루|
|            광화문|
|            광흥창|
|              교대|
|              구의|
|              군자|
|          굽은다리|
|            낙성대|
|        남부터미널|
|      남한산성입구|
|            녹사평|
|              대림|
|              대흥|
|동대문역사문화공원|
|              동작|
|          둔촌오륜|
|          마곡나루|
|          몽촌토성|
|              미사|
|              미아|
|            봉화산|
|              사리|
|              산곡|
|              삼성|
|              삼전|
|              상봉|
|            상월곡|
|              새절|
|        서울대입구|
|              석남|
|              석촌|
|          석촌고분|
|      성신여대입구|
|          송파나루|
|              수유|
|          숙대입구|
|        숭실대입구|
|              신정|
|              신창|
|              쌍용|
|            아차산|
|              안암|
|              야목|
|              양재|
|      어린이대공원|
|    

In [ ]:
# 419민주묘지 같이 병기역명이 아닌역들은 따로 확인을 해줘야한다.
df.select('역명').distinct().orderBy('역명').show(600)

+-----------------------------+
|                         역명|
+-----------------------------+
|                 4.19민주묘지|
|                         가능|
|                     가락시장|
|               가산디지털단지|
|                         가양|
|                       가오리|
|                         가좌|
|                       가천대|
|                         가평|
|                         간석|
|                         갈매|
|                         강남|
|                     강남구청|
|                         강동|
|                     강동구청|
|                         강매|
|           강변(동서울터미널)|
|                         강촌|
|                         개롱|
|                         개봉|
|                       개포동|
|                         개화|
|                       개화산|
|                         거여|
|                     건대입구|
|                         검암|
|                     경기광주|
|                     경마공원|
|         경복궁(정부서울청사)|
|                     경찰병원|
|                         계양|
|             

<h1>여러 데이터프레임의 역명 통일하기</h1>
우리는 지도에 표시한 역들에 대한 정보가 필요하므로 ordered에 left outer join을 실시한다.


In [ ]:
# 원본데이터 확인용 데이터프레임
origin = spark.read.parquet('/content/drive/MyDrive/data/all_metro')

In [ ]:
# 필요한 역을 조회(필요할때마다 수시로 조회했음)
origin.where(F.col('역명')=='계양').show()

+----------+----------+----+------------+------------+--------+
|  사용일자|    노선명|역명|승차총승객수|하차총승객수|이용객수|
+----------+----------+----+------------+------------+--------+
|2018-01-11|  공항철도|계양|       11127|       10299|   21426|
|2018-01-12|  공항철도|계양|       11217|       10333|   21550|
|2018-01-13|  공항철도|계양|        8678|        8073|   16751|
|2018-01-14|  공항철도|계양|        6180|        6113|   12293|
|2018-01-15|  공항철도|계양|       11795|       10832|   22627|
|2018-01-16|  공항철도|계양|       11796|       10898|   22694|
|2018-01-17|  공항철도|계양|       11769|       10860|   22629|
|2018-01-18|  공항철도|계양|       11877|       11019|   22896|
|2018-01-19|  공항철도|계양|       12186|       11355|   23541|
|2018-01-20|  공항철도|계양|        9018|        8242|   17260|
|2018-01-21|  공항철도|계양|        6499|        6480|   12979|
|2018-01-22|  공항철도|계양|       11828|       11041|   22869|
|2018-01-23|경의중앙선|계양|           1|           0|       1|
|2018-01-23|  공항철도|계양|       11379|       10640|   22019|
|2018-01-24|  공항철도|계양|

In [ ]:
test = ordered.join(station_locate, on='역명', how='left')

In [ ]:
# ordered에는 있고 station_locate에는 없는 역을 확인
test.select('역명', '노선명').where(F.col('lat').isNull()==True).distinct().orderBy('노선명').show()

+----+------+
|역명|노선명|
+----+------+
+----+------+



In [ ]:
# 이용자수를 기록한 데이터프레임은 병기역명들이 표기되어있다. 그러나 test에는 병기역명이 없다.
test.where(F.col('노선명')=='2호선').orderBy('sub노선명').show()

+------------------+------+---------+----+---------+----------+
|              역명|노선명|sub노선명|순번|      lat|       lng|
+------------------+------+---------+----+---------+----------+
|              시청| 2호선|    2호선| 201|37.564718|126.977108|
|        을지로입구| 2호선|    2호선| 202|37.566014|126.982618|
|         을지로3가| 2호선|    2호선| 203|37.566295| 126.99191|
|         을지로4가| 2호선|    2호선| 204|37.566941|126.998079|
|동대문역사문화공원| 2호선|    2호선| 205|37.565138|127.007896|
|              신당| 2호선|    2호선| 206|37.565972| 127.01782|
|          상왕십리| 2호선|    2호선| 207|37.564354|127.029354|
|            왕십리| 2호선|    2호선| 208|37.561533|127.037732|
|            한양대| 2호선|    2호선| 209|37.555273|127.043655|
|              뚝섬| 2호선|    2호선| 210|37.547184|127.047367|
|              성수| 2호선|    2호선| 211|37.544581|127.055961|
|          건대입구| 2호선|    2호선| 212|37.540693| 127.07023|
|              구의| 2호선|    2호선| 213|37.537077|127.085916|
|              강변| 2호선|    2호선| 214|37.535095|127.094681|
|          잠실나루| 2호선|    2

두 데이터프레임 모두 병기역명 표기가 안되어있다 변환이 필요하다.

In [ ]:
# 해당하는 역이 많이므로 딕셔너리를 활용하려한다.(수작업)
duel_name = {'충정로': '충정로(경기대입구)', '안암': '안암(고대병원앞)', '천호': '천호(풍납토성)',
             '신창': '신창(순천향대)', '봉화산': '봉화산(서울의료원)', '광화문': '광화문(세종문화회관)',
             '마곡나루': '마곡나루(서울식물원)', '어린이대공원': '어린이대공원(세종대)', '경복궁': '경복궁(정부서울청사)',
             '상봉': '상봉(시외버스터미널)', '왕십리': '왕십리(성동구청)', '교대': '교대(법원.검찰청)',
             '남한산성입구': '남한산성입구(성남법원.검찰청)', '수유': '수유(강북구청)', '월드컵경기장': '월드컵경기장(성산)',
             '쌍용': '쌍용(나사렛대)', '청량리': '청량리(서울시립대입구)', '숭실대입구': '숭실대입구(살피재)',
             '총신대입구': '총신대입구(이수)', '동작': '동작(현충원)', '새절': '새절(신사)', '강변': '강변(동서울터미널)',
             '몽촌토성': '몽촌토성(평화의문)', '아차산': '아차산(어린이대공원후문)', '고려대': '고려대(종암)', '증산': '증산(명지대앞)',
             '광흥창': '광흥창(서강)', '서울대입구': '서울대입구(관악구청)', '군자': '군자(능동)', '오목교': '오목교(목동운동장앞)',
             '신정': '신정(은행정)', '잠실': '잠실(송파구청)', '미아': '미아(서울사이버대학)', '삼성': '삼성(무역센터)',
             '대림': '대림(구로구청)', '회현': '회현(남대문시장)', '구의': '구의(광진구청)', '용마산': '용마산(용마폭포공원)',
             '남부터미널': '남부터미널(예술의전당)', '화랑대': '화랑대(서울여대입구)', '녹사평': '녹사평(용산구청)',
             '용두': '용두(동대문구청)', '대흥': '대흥(서강대앞)', '낙성대': '낙성대(강감찬)', '양재': '양재(서초구청)',
             '상월곡': '상월곡(한국과학기술연구원)', '숙대입구': '숙대입구(갈월)', '광나루': '광나루(장신대)',
             '온수': '온수(성공회대입구)', '올림픽공원': '올림픽공원(한국체대)', '한성대입구': '한성대입구(삼선교)',
             '이촌': '이촌(국립중앙박물관)', '동대문역사문화공원': '동대문역사문화공원(DDP)', '공릉': '공릉(서울과학기술대)',
             '성신여대입구': '성신여대입구(돈암)', '굽은다리': '굽은다리(강동구민회관앞)', '월곡': '월곡(동덕여대)', '흑석': '흑석(중앙대입구)'}

In [ ]:
duel_name_list = []
for key in duel_name:
    duel_name_list.append(key)

In [ ]:
# test의 역명을 바꿔줌
mapping_expr = F.create_map([F.lit(x) for x in chain(*duel_name.items())])
test = test.withColumn("역명", F.when(test.역명.isin(duel_name_list), mapping_expr[F.col("역명")]).otherwise(test.역명))

In [ ]:
station_locate

DataFrame[역명: string, lat: string, lng: string]

삭제해야 할 노선(최근에 개통하여 620일 동안의 평균 이용자를 구할 수 없음)<br>
강일, 고색, 둔촌오륜, 미사, 사리, 산곡, 삼전, 석촌고분, 송파나루, 야목, 어천, 오목천, 임진강, 중앙보훈병원, 탕정, 하남검단산, 하남시청, 하남풍산, 한성백제<br>
이름을 바꿔야 할 노선<br>
419민주묘지 -> 4.19민주묘지

In [ ]:
test2 = test.join(df2, on='역명', how='left')

In [ ]:
# df2에 누락된 역이 있는지 확인
test2.where(F.col('lat').isNull()==True).orderBy('역명').show()

+----+------+---------+----+---+---+------------+----------------+
|역명|노선명|sub노선명|순번|lat|lng|이용자수차이|일간이용자수차이|
+----+------+---------+----+---+---+------------+----------------+
+----+------+---------+----+---+---+------------+----------------+



In [ ]:
# join을 반대로 해서 역으로 df2에는 있지만 test에는 없는 역이 있는지 확인
df2.join(test, on='역명', how='left').where(F.col('lat').isNull()==True).show()

+----+------------+----------------+------+---------+----+---+---+
|역명|이용자수차이|일간이용자수차이|노선명|sub노선명|순번|lat|lng|
+----+------------+----------------+------+---------+----+---+---+
+----+------------+----------------+------+---------+----+---+---+



In [ ]:
# 그래프 작업에 용이하게 sub노선명 기준으로 정렬한다.
result = test2.orderBy('sub노선명')

In [ ]:
result.show()

+----------+------+---------+----+---------+----------+------------+----------------+
|      역명|노선명|sub노선명|순번|      lat|       lng|이용자수차이|일간이용자수차이|
+----------+------+---------+----+---------+----------+------------+----------------+
|    소요산| 1호선|    1호선| 100|  37.9481|127.061034|    -1739870|           -2806|
|    동두천| 1호선|    1호선| 101|37.927878| 127.05479|     -942065|           -1519|
|      보산| 1호선|    1호선| 102|37.913702|127.057277|     -649745|           -1047|
|동두천중앙| 1호선|    1호선| 103|37.901885|127.056482|    -1464693|           -2362|
|      지행| 1호선|    1호선| 104|37.892334|127.055716|    -2745862|           -4428|
|      덕정| 1호선|    1호선| 105|37.843188|127.061277|    -2721122|           -4388|
|      덕계| 1호선|    1호선| 106|37.818486|127.056486|      -66368|            -107|
|      양주| 1호선|    1호선| 107|37.774381|127.044708|    -3270023|           -5274|
|      녹양| 1호선|    1호선| 108| 37.75938|127.042292|    -1494609|           -2410|
|      가능| 1호선|    1호선| 109|37.748577|127.044213|  

In [ ]:
# 확인
result.where(F.col('역명')=='시청').show()

+----+------+---------+----+---------+----------+------------+----------------+
|역명|노선명|sub노선명|순번|      lat|       lng|이용자수차이|일간이용자수차이|
+----+------+---------+----+---------+----------+------------+----------------+
|시청| 1호선|    1호선| 132|37.564718|126.977108|   -21519048|          -34708|
|시청| 2호선|    2호선| 201|37.564718|126.977108|   -21519048|          -34708|
|시청| 2호선|    2호선| 244|37.564718|126.977108|   -21519048|          -34708|
+----+------+---------+----+---------+----------+------------+----------------+



In [ ]:
# 확인
result.where(F.col('역명')=='양평').show()

+----+----------+----------+----+---------+----------+------------+----------------+
|역명|    노선명| sub노선명|순번|      lat|       lng|이용자수차이|일간이용자수차이|
+----+----------+----------+----+---------+----------+------------+----------------+
|양평|경의중앙선|경의중앙선| 135|37.492832|127.491814|    -1527195|           -2463|
+----+----------+----------+----+---------+----------+------------+----------------+



In [ ]:
# 이러면 되나? 하고 테스트해본건데 작업이 잘되서 그냥 저장함
result.write.parquet('/content/drive/MyDrive/data/contrast_metro_user/ordered_line_difference.parquet')

In [ ]:
# 저장이 잘되었는지 확인
df = spark.read.format("parquet").option("header", True).load("/content/drive/MyDrive/data/contrast_metro_user/ordered_line_difference.parquet")
df.head()

Row(역명='소요산', 노선명='1호선', sub노선명='1호선', 순번='100', lat='37.9481', lng='127.061034', 이용자수차이=-1739870, 일간이용자수차이=-2806)

In [ ]:
# readme에 써야해서 row개수 세봄
df.count()

619